In [1]:
from graph import *
from models import *
from metrics import *

import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import plotly.graph_objects as go

In [2]:
from copy import deepcopy

In [3]:
# load data from the spreadsheet which defines the structure of the workflow,
# as well as the parameters for data rates, efficiency, data reduction, and classifier performance
run3_system = dataframes_from_spreadsheet("cms_system_60.xlsx")
run5_system = dataframes_from_spreadsheet("cms_system_200.xlsx")

In [4]:
run5_system.detectors

,Category,Detector,Data (bytes),Sample Rate,Compression,Link Efficiency (J/bit),Op Efficiency (J/op),PU 200
0,Tracking,Inner Tracker,1440000,40000000,0,2.220000e-11,0,1.440
1,Tracking,Outer Tracker PS,720000,40000000,0,2.220000e-11,0,0.720
2,Tracking,Outer Tracker 2S,430000,40000000,0,2.220000e-11,0,0.430
3,Tracking,Track Finder TPG,10000,40000000,0,2.220000e-11,0,0.010
4,Timing,MIP Timing BTL,240000,40000000,0,2.220000e-11,0,0.240
5,Timing,MIP Timing ETL,440000,40000000,0,2.220000e-11,0,0.440
6,Calorimetry,ECAL Barrel,600000,40000000,0,2.220000e-11,0,0.600
7,Calorimetry,HCAL Barrel,240000,40000000,0,2.220000e-11,0,0.240
8,Calorimetry,HCAL HO,30000,40000000,0,2.220000e-11,0,0.030
9,Calorimetry,HCAL HF,60000,40000000,0,2.220000e-11,0,0.060


In [5]:
#import the data predicting wall time scaling by pileup
scaling = pd.read_excel("wall time scaling.xlsx", sheet_name="Data")
#fit a polynomial to this data for CPU and GPU runtimes
fit_poly = lambda x, k3, k2, k1: k3 * x ** 3 + k2 * x ** 2 + k1 * x
k, cv = curve_fit(fit_poly, scaling["Size"], scaling["Wall Time"])
k_gpu, cv_gpu = curve_fit(fit_poly, scaling["Size"], scaling["Wall Time GPU"])

In [6]:
#define a dictionary with functions defining the scaling of trigger runtimes with incoming data
funcs = {"Global": lambda x: fit_poly(x, *k), "Intermediate": lambda x: x / 2.0e6}
funcs_gpu = {"Global": lambda x: fit_poly(x, *k_gpu), "Intermediate": lambda x: x / 2.0e6}

In [7]:
baseline_r3 = construct_graph(run3_system.detectors, run3_system.triggers, run3_system.globals, funcs)

/Users/wilkie/code/system_flow/classifier.py:197: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  fit = lambda l: np.abs(self.egamma_rate - quad(lambda x: self.exp_dist(x, l) * interpolator(x), np.min(xs), np.max(xs))[0])


In [8]:
baseline = construct_graph(run5_system.detectors, run5_system.triggers, run5_system.globals, funcs)

/Users/wilkie/code/system_flow/classifier.py:197: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  fit = lambda l: np.abs(self.egamma_rate - quad(lambda x: self.exp_dist(x, l) * interpolator(x), np.min(xs), np.max(xs))[0])


In [9]:
baseline.nodes["Intermediate"]

{'type': 'processor',
 'reduction ratio': 53.3,
 'classifier': <classifier.L1TClassifier at 0x307ad0c70>,
 'data reduction': 1.0,
 'op efficiency': 0.003,
 'sample data': 260000,
 'complexity': <function __main__.<lambda>(x)>,
 'global ratio': 5330.0,
 'message size': 8425000.0,
 'ops': 4.2125,
 'input rate': 39999999,
 'error matrix': array([[0.98132, 0.5205 ],
        [0.01868, 0.4795 ]]),
 'contingency': array([[39245435,     3905],
        [  747059,     3598]]),
 'discards': array([39245435,     3905]),
 'output rate': 750657,
 'energy': 0.012637500000000001,
 'power': 505499.98736250005}

In [10]:
baseline.nodes["Intermediate"]["energy"] * np.sum(baseline.nodes["Intermediate"]["discards"])

496013.53425

In [11]:
a1 = list(baseline.predecessors("Intermediate"))

In [12]:
a1

['Tracking', 'Timing', 'Calorimetry', 'Muon']

In [13]:
list(baseline.successors("Intermediate"))

['Global']

In [14]:
"Intermediate" in baseline.nodes.keys()

True

In [15]:
"""
Return the amount of energy expended by the system to reach the current node
"""
def upstream_energy(graph, node):
    def get_energy(node):
        if "energy" in node.keys():
            return node["energy"]
        else:
            return 0.0
    
    def traverse(start):
        up = list(graph.predecessors(start))

        if len(up) == 0:
            return get_energy(graph.nodes[start])
        else:
            return get_energy(graph.nodes[start]) + functools.reduce(lambda x, y: x + y, map(traverse, up))
    
    return traverse(node)

In [16]:
upstream_energy(baseline, "Intermediate")

0.012637500000000001

In [17]:
baseline.graph["Root Node"]

'Disk'

In [18]:
baseline.nodes

NodeView(('Inner Tracker', 'Outer Tracker PS', 'Outer Tracker 2S', 'Track Finder TPG', 'MIP Timing BTL', 'MIP Timing ETL', 'ECAL Barrel', 'HCAL Barrel', 'HCAL HO', 'HCAL HF', 'HGCAL', 'HGCAL TPG Stage1', 'HGCAL TPG Stage2', 'Muon DT', 'Muon CSC', 'Muon GEM GE1', 'Muon GEM GE2', 'Muon GEM ME0', 'Muon RPC', 'Tracking', 'Timing', 'Calorimetry', 'Muon', 'Intermediate', 'Global', 'Disk'))

In [19]:
upstream_energy(baseline, "Disk")

446.05484938484136

In [20]:
def quantify_error_cost(graph):
    #the cost of a true positive is the cost to get to the final node
    positive = upstream_energy(graph, graph.graph["Root Node"])
    #the cost of a true negative is the average energy for a discarded message
    classifiers = active_classifiers(graph)
    energy = [upstream_energy(graph, c) for c in classifiers]
    negatives = [np.sum(graph.nodes[c]["discards"]) for c in classifiers]
    negative = np.average(energy, weights=negatives)

    return (negative, positive)

In [21]:
#quantify_error_cost(baseline)

In [22]:
baseline.nodes["Inner Tracker"]["global ratio"]

5330.0

In [23]:
(5330 * 8.3) / 446

99.19058295964128

In [24]:
5330 * 8.3

44239.00000000001

In [25]:
baseline.nodes["Global"]["discards"]

array([742891,    258])

In [26]:
baseline.nodes["Intermediate"]["energy"] * baseline.nodes["Intermediate"]["output rate"]

9486.427837500001

In [27]:
gpu = construct_graph(run5_system.detectors, run5_system.triggers, run5_system.globals, funcs_gpu)

/Users/wilkie/code/system_flow/classifier.py:197: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  fit = lambda l: np.abs(self.egamma_rate - quad(lambda x: self.exp_dist(x, l) * interpolator(x), np.min(xs), np.max(xs))[0])


In [28]:
gpu.nodes["Disk"]

{'type': 'storage',
 'reduction ratio': 1.0,
 'classifier': <classifier.DummyClassifier at 0x307b221a0>,
 'data reduction': 1.0,
 'op efficiency': 0.0,
 'sample data': 0,
 'complexity': <function graph.triggers.<locals>.<lambda>(x)>,
 'global ratio': 1.0,
 'message size': 8425000.0,
 'ops': 8425000.0,
 'input rate': 7506,
 'error matrix': array([[0., 0.],
        [1., 1.]]),
 'contingency': array([[   0,    0],
        [4138, 3368]]),
 'discards': array([0, 0]),
 'output rate': 7506,
 'energy': 0.0,
 'power': 0.0}

In [29]:
baseline_2 = deepcopy(baseline)
baseline_2.nodes["Intermediate"]["reduction ratio"] = 400
baseline_2 = update_throughput(baseline_2)


In [30]:
l1t = deepcopy(baseline)
l1t.nodes["Intermediate"]["classifier"].skill_boost = 0.40
l1t = update_throughput(l1t)

In [31]:
smpx = deepcopy(baseline)
smpx.nodes["Inner Tracker"]["sample data"] *= (1 - 0.54)
smpx = update_throughput(smpx)

In [32]:
gpu_smpx = deepcopy(gpu)
gpu_smpx.nodes["Inner Tracker"]["sample data"] *= (1 - 0.54)
gpu_smpx = update_throughput(gpu_smpx)

In [33]:
gpu_l1t = deepcopy(gpu)
gpu_l1t.nodes["Intermediate"]["classifier"].skill_boost = 0.40
gpu_l1t = update_throughput(gpu_l1t)

In [34]:
smpx_l1t = deepcopy(baseline)
smpx_l1t.nodes["Inner Tracker"]["sample data"] *= (1 - 0.54)
smpx_l1t.nodes["Intermediate"]["classifier"].skill_boost = 0.40
smpx_l1t = update_throughput(smpx_l1t)

In [35]:
gpu_smpx_l1t = deepcopy(gpu)
gpu_smpx_l1t.nodes["Intermediate"]["classifier"].skill_boost = 0.40
gpu_smpx_l1t.nodes["Inner Tracker"]["sample data"] *= (1 - 0.54)
gpu_smpx_l1t = update_throughput(gpu_smpx_l1t)

In [36]:
np.sum(gpu.graph["performance"][:,1])

7502

In [37]:
gpu.nodes["Intermediate"]

{'type': 'processor',
 'reduction ratio': 53.3,
 'classifier': <classifier.L1TClassifier at 0x307b23850>,
 'data reduction': 1.0,
 'op efficiency': 0.003,
 'sample data': 260000,
 'complexity': <function __main__.<lambda>(x)>,
 'global ratio': 5330.0,
 'message size': 8425000.0,
 'ops': 4.2125,
 'input rate': 39999999,
 'error matrix': array([[0.98132, 0.51624],
        [0.01868, 0.48376]]),
 'contingency': array([[39245435,     3873],
        [  747059,     3630]]),
 'discards': array([39245435,     3873]),
 'output rate': 750689,
 'energy': 0.012637500000000001,
 'power': 505499.98736250005}

In [38]:
has_classifier(gpu.nodes["Intermediate"])

True

In [40]:
ac = active_classifiers(gpu)

In [41]:
[downstream_classifier(gpu, c) for c in ac]

[True, False]

In [45]:
gpu.nodes["Intermediate"]["contingency"]

array([[39245435,     3873],
       [  747059,     3630]])

In [44]:
gpu.nodes["Global"]["contingency"]

array([[742920,    261],
       [  4138,   3368]])

In [42]:
pipeline_contingency(gpu)

array([[39988355,     4134],
       [    4138,     3368]])

In [101]:
precision(gpu.nodes["Intermediate"]["contingency"])

0.004763966508356259

In [102]:
recall(gpu.nodes["Intermediate"]["contingency"])

0.476609356257497

In [103]:
gpu.graph

{'globals':    Year
 0  2032,
 'Root Node': 'Disk',
 'link power': 124601.62426700002,
 'op power': 167912946.6070844,
 'performance': array([[39988307,     4183],
        [    4186,     3319]])}

In [19]:
def extract_results(graph):
    power = (graph.graph["op power"] + graph.graph["link power"]) / density_scale_model(2032)
    confusion = graph.graph["performance"]
    acc = precision(confusion)
    rec = recall(confusion)
    f1 = f1_score(confusion)
    prod = f1 * np.sum(get_passed(confusion)) / power


    return power, acc, rec, f1, prod

In [20]:
conditions = [baseline_r3, baseline_2, baseline]

In [21]:
pileup = np.array([60, 200, 200])[:,np.newaxis]
rejection = np.array([400, 400, 53])[:,np.newaxis]

In [22]:
pileup.shape

(3, 1)

In [23]:
cond_results = np.stack([extract_results(g) for g in conditions])

In [25]:
cond_results = np.concatenate((pileup, rejection, cond_results), axis=1)

In [26]:
df2 = pd.DataFrame(cond_results, columns = ["Pileup", "L1T Reduction Ratio", "Power (W)", "Accuracy (%)", "Recall (%)", "F1 Score (%)", "Productivity (Relevant Samples/J)"])

In [27]:
df2

,Pileup,L1T Reduction Ratio,Power (W),Accuracy (%),Recall (%),F1 Score (%),Productivity (Relevant Samples/J)
0,60.0,400.0,3.223540e+05,0.261477,0.262525,0.262000,0.000814
1,200.0,400.0,6.703782e+06,0.271310,0.261523,0.266327,0.000038
2,200.0,53.0,5.161370e+07,0.448042,0.448280,0.448161,0.000065


In [28]:
extract_results(baseline_2)[-1] * 1000

0.03821814811904678

In [29]:
extract_results(baseline)[-1] * 1000

0.0651744907746219

In [30]:
extract_results(baseline_r3)[-1] * 1000

0.8143965102210354

In [31]:
extract_results(baseline_2)[-1] * 1000

0.03821814811904678

In [32]:
extract_results(baseline)[-1] * 1000

0.0651744907746219

In [33]:
all_graphs = [baseline_r3, baseline, gpu, l1t, smpx, gpu_l1t, smpx_l1t, gpu_smpx, gpu_smpx_l1t]

In [34]:
pileup = np.array([[60, 200, 200, 200, 200, 200, 200, 200, 200,],])
rejection = np.array([[400, 53, 53, 53, 53, 53, 53, 53, 53],])
has_gpu = [False, False, True, False, False, True, False, True, True]
has_smpx = [False, False, False, False, True, False, True, True, True]
has_l1t = [False, False, False, True, False, True, True, False, True]

In [35]:
results = np.stack([extract_results(g) for g in all_graphs])

In [36]:
results

array([[3.22354034e+05, 2.61477046e-01, 2.62525050e-01, 2.62000000e-01,
        8.14396510e-04],
       [5.16136955e+07, 4.48041567e-01, 4.48280459e-01, 4.48160981e-01,
        6.51744908e-05],
       [2.58536388e+07, 4.42238508e-01, 4.42415356e-01, 4.42326914e-01,
        1.28402177e-04],
       [5.15966761e+07, 8.39818763e-01, 8.40042655e-01, 8.39930694e-01,
        1.22155930e-04],
       [4.08461281e+07, 4.48041567e-01, 4.48280459e-01, 4.48160981e-01,
        8.23553291e-05],
       [2.58190851e+07, 8.39893262e-01, 8.39109571e-01, 8.39501234e-01,
        2.43698091e-04],
       [4.08326629e+07, 8.39818763e-01, 8.40042655e-01, 8.39930694e-01,
        1.54357798e-04],
       [2.04657045e+07, 4.42238508e-01, 4.42415356e-01, 4.42326914e-01,
        1.62206167e-04],
       [2.04383669e+07, 8.39893262e-01, 8.39109571e-01, 8.39501234e-01,
        3.07855406e-04]])

In [37]:
pileup.shape

(1, 9)

In [38]:
rejection.shape

(1, 9)

In [39]:
results = np.concatenate((pileup, rejection, np.transpose(results)), axis=0)

In [40]:
results

array([[6.00000000e+01, 2.00000000e+02, 2.00000000e+02, 2.00000000e+02,
        2.00000000e+02, 2.00000000e+02, 2.00000000e+02, 2.00000000e+02,
        2.00000000e+02],
       [4.00000000e+02, 5.30000000e+01, 5.30000000e+01, 5.30000000e+01,
        5.30000000e+01, 5.30000000e+01, 5.30000000e+01, 5.30000000e+01,
        5.30000000e+01],
       [3.22354034e+05, 5.16136955e+07, 2.58536388e+07, 5.15966761e+07,
        4.08461281e+07, 2.58190851e+07, 4.08326629e+07, 2.04657045e+07,
        2.04383669e+07],
       [2.61477046e-01, 4.48041567e-01, 4.42238508e-01, 8.39818763e-01,
        4.48041567e-01, 8.39893262e-01, 8.39818763e-01, 4.42238508e-01,
        8.39893262e-01],
       [2.62525050e-01, 4.48280459e-01, 4.42415356e-01, 8.40042655e-01,
        4.48280459e-01, 8.39109571e-01, 8.40042655e-01, 4.42415356e-01,
        8.39109571e-01],
       [2.62000000e-01, 4.48160981e-01, 4.42326914e-01, 8.39930694e-01,
        4.48160981e-01, 8.39501234e-01, 8.39930694e-01, 4.42326914e-01,
        8.3

In [41]:
df = pd.DataFrame(results.transpose(), columns = ["Pileup", "L1T Reduction Ratio", "Power (W)", "Accuracy (%)", "Recall (%)", "F1 Score (%)", "Productivity (Relevant Samples/J)"])

In [42]:
df["GPU HLT"] = has_gpu
df["L1T Tracking"] = has_l1t
df["Smart Sensors"] = has_smpx

In [43]:
df

,Pileup,L1T Reduction Ratio,Power (W),Accuracy (%),Recall (%),F1 Score (%),Productivity (Relevant Samples/J),GPU HLT,L1T Tracking,Smart Sensors
0,60.0,400.0,3.223540e+05,0.261477,0.262525,0.262000,0.000814,False,False,False
1,200.0,53.0,5.161370e+07,0.448042,0.448280,0.448161,0.000065,False,False,False
2,200.0,53.0,2.585364e+07,0.442239,0.442415,0.442327,0.000128,True,False,False
3,200.0,53.0,5.159668e+07,0.839819,0.840043,0.839931,0.000122,False,True,False
4,200.0,53.0,4.084613e+07,0.448042,0.448280,0.448161,0.000082,False,False,True
5,200.0,53.0,2.581909e+07,0.839893,0.839110,0.839501,0.000244,True,True,False
6,200.0,53.0,4.083266e+07,0.839819,0.840043,0.839931,0.000154,False,True,True
7,200.0,53.0,2.046570e+07,0.442239,0.442415,0.442327,0.000162,True,False,True
8,200.0,53.0,2.043837e+07,0.839893,0.839110,0.839501,0.000308,True,True,True


In [44]:
df.iloc[1:]

,Pileup,L1T Reduction Ratio,Power (W),Accuracy (%),Recall (%),F1 Score (%),Productivity (Relevant Samples/J),GPU HLT,L1T Tracking,Smart Sensors
1,200.0,53.0,5.161370e+07,0.448042,0.448280,0.448161,0.000065,False,False,False
2,200.0,53.0,2.585364e+07,0.442239,0.442415,0.442327,0.000128,True,False,False
3,200.0,53.0,5.159668e+07,0.839819,0.840043,0.839931,0.000122,False,True,False
4,200.0,53.0,4.084613e+07,0.448042,0.448280,0.448161,0.000082,False,False,True
5,200.0,53.0,2.581909e+07,0.839893,0.839110,0.839501,0.000244,True,True,False
6,200.0,53.0,4.083266e+07,0.839819,0.840043,0.839931,0.000154,False,True,True
7,200.0,53.0,2.046570e+07,0.442239,0.442415,0.442327,0.000162,True,False,True
8,200.0,53.0,2.043837e+07,0.839893,0.839110,0.839501,0.000308,True,True,True


In [45]:
df.iloc[1:]["Productivity (Relevant Samples/J)"] * 1000

1    0.065174
2    0.128402
3    0.122156
4    0.082355
5    0.243698
6    0.154358
7    0.162206
8    0.307855
Name: Productivity (Relevant Samples/J), dtype: float64

In [46]:
df["Productivity (Relevant Samples/J)"] * 1e3

0    0.814397
1    0.065174
2    0.128402
3    0.122156
4    0.082355
5    0.243698
6    0.154358
7    0.162206
8    0.307855
Name: Productivity (Relevant Samples/J), dtype: float64

In [47]:
df.to_excel("experimental_table.xlsx", index=False)

In [48]:
density_scale_model(2032)

6.499570514329353